In [63]:
# 사진 갯수
from glob import glob
pngs = sorted(glob('unzip/*/*/*/*/*.png')) 
len(pngs)

473351

In [64]:
# 파일명 파싱
from tqdm.auto import tqdm
print(pngs[-1])
%time names = [e.split('/')[-1].split('.')[0].split('_') for e in pngs]
J = lambda n, i: '_'.join(n[:i])
A = lambda n: [J(n,1), J(n,2), J(n,3) , J(n,4), J(n,5)]
%time rows = [A(n) + [p] for n, p in zip(names, pngs)]
rows[-2:]

unzip/OUT/H00646/SN4/100810/OUT_H00646_SN4_100810_20397.png
CPU times: user 440 ms, sys: 32.5 ms, total: 473 ms
Wall time: 469 ms
CPU times: user 1.53 s, sys: 71.6 ms, total: 1.6 s
Wall time: 1.6 s


[['OUT',
  'OUT_H00646',
  'OUT_H00646_SN4',
  'OUT_H00646_SN4_100810',
  'OUT_H00646_SN4_100810_20394',
  'unzip/OUT/H00646/SN4/100810/OUT_H00646_SN4_100810_20394.png'],
 ['OUT',
  'OUT_H00646',
  'OUT_H00646_SN4',
  'OUT_H00646_SN4_100810',
  'OUT_H00646_SN4_100810_20397',
  'unzip/OUT/H00646/SN4/100810/OUT_H00646_SN4_100810_20397.png']]

In [65]:
# 데이터 프레임으로 변환
import pandas as pd
df = pd.DataFrame(rows, columns=['inout', 'H', 'SN', 'ID', 'PNG', 'path'])
df.tail(5)

,inout,H,SN,ID,PNG,path
473346,OUT,OUT_H00646,OUT_H00646_SN4,OUT_H00646_SN4_100810,OUT_H00646_SN4_100810_20385,unzip/OUT/H00646/SN4/100810/OUT_H00646_SN4_100...
473347,OUT,OUT_H00646,OUT_H00646_SN4,OUT_H00646_SN4_100810,OUT_H00646_SN4_100810_20388,unzip/OUT/H00646/SN4/100810/OUT_H00646_SN4_100...
473348,OUT,OUT_H00646,OUT_H00646_SN4,OUT_H00646_SN4_100810,OUT_H00646_SN4_100810_20391,unzip/OUT/H00646/SN4/100810/OUT_H00646_SN4_100...
473349,OUT,OUT_H00646,OUT_H00646_SN4,OUT_H00646_SN4_100810,OUT_H00646_SN4_100810_20394,unzip/OUT/H00646/SN4/100810/OUT_H00646_SN4_100...
473350,OUT,OUT_H00646,OUT_H00646_SN4,OUT_H00646_SN4_100810,OUT_H00646_SN4_100810_20397,unzip/OUT/H00646/SN4/100810/OUT_H00646_SN4_100...


In [66]:
# 컬럼별  nuique 통계
df.describe().loc[['unique']]

,inout,H,SN,ID,PNG,path
unique,2,102,401,3981,473351,473351


In [109]:
# H(uman) 샘플 보기
import ipyplot
h = df['H'].sample(n=1).values[0]
print(h)
sample_h = df.query('H == @h').sample(n=10)['path'].values
ipyplot.plot_images(sample_h, img_width=30, force_b64=True)

OUT_H00594


In [153]:
# SN 샘플 보기
sn = df.query('H == @h')['SN'].sample(n=2).values[:2]
print(sn)
sample_sn = df.query('SN == @sn[0]').sample(10)['path'].values
ipyplot.plot_images(sample_sn, img_width=30, force_b64=True)

sample_sn = df.query('SN == @sn[1]').sample(10)['path'].values
ipyplot.plot_images(sample_sn, img_width=30, force_b64=True)

['OUT_H00594_SN4' 'OUT_H00594_SN1']


In [161]:
# ID 샘플 보기
id_ = df.query('H == @h and SN == @sn[0]')['ID'].sample(n=2).values[:2]
print(id_)
sample_id_0 = df.query('ID == @id_[0]')['path'].sample(n=40).values[::4]
ipyplot.plot_images(sample_id_0, img_width=30, force_b64=True)

sample_id_1 = df.query('ID == @id_[1]')['path'].sample(n=40).values[::4]
ipyplot.plot_images(sample_id_1, img_width=30, force_b64=True)

['OUT_H00594_SN4_100706' 'OUT_H00594_SN4_100701']


In [160]:
from torchreid.utils import FeatureExtractor

extractor = FeatureExtractor(
    model_name='osnet_ain_x1_0',
    model_path='./weights/osnet_ain_x1_0_reid.pth',
    device='cuda'
)

Successfully loaded imagenet pretrained weights from "/home/kts2/.cache/torch/checkpoints/osnet_ain_x1_0_imagenet.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']
Model: osnet_ain_x1_0
- params: 2,193,616
- flops: 978,878,352
Successfully loaded pretrained weights from "./weights/osnet_ain_x1_0_reid.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']


In [170]:
ebds_0 = extractor(list(sample_id_0))
ebds_1 = extractor(list(sample_id_1))
ebds_0.shape, ebds_1.shape

(torch.Size([10, 512]), torch.Size([10, 512]))

In [171]:
import random
import torch.nn.functional as F


# 사잇각의 평균, 표준편차, 최대, 최소
def mean_diff(ebds):
    ebds = F.normalize(ebds, p=2, dim=-1)
    ebds_t = ebds.transpose(1, 0)
    dots = ebds@ebds_t
    diff = 1 - dots
    mean = diff.sum()/(len(ebds)*len(ebds) - len(ebds))
    r = [mean, diff.max(), diff.std(), diff.clone().fill_diagonal_(1).min()]
    r = [e.cpu().numpy().round(3) for e in r]
    return r

In [172]:
# 사잇각 평균, max, std, min
sts_0 = [mean_diff(e) for e in ebds_0]
sts_1 = [mean_diff(e) for e in ebds_1]

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)